<a href="https://colab.research.google.com/github/Luana-Camille/-ia-grafos-Luana-Camille/blob/main/ia_grafos_buscas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lista de Exercícios N2**

Tema: Grafos e Algoritmos de Busca (Dijkstra, A*, em-ordem, pré-ordem e pós-ordem)

Disciplina: Inteligência Artificial\
Ambiente: Google Colab\
Entrega: via repositório individual no GitHub

**Nome completo: Luana Camille da Silva Lima**\
**Matricula: 2138115**

**Regras Gerais**

- Trabalho individual.

- Linguagem: Python 3 (Google Colab).

- É permitido usar: heapq, numpy, matplotlib, dataclasses.

- Proibido usar funções prontas de shortest path (networkx.shortest_path, scipy.sparse.csgraph.dijkstra, etc.).

O Notebook (.ipynb) deve conter:

- Identificação (nome, turma, link do GitHub)

- Código, testes e reflexões

- Seções organizadas conforme o roteiro abaixo.

**Parte A — Dijkstra (Caminho Mínimo em Grafos Ponderados Positivos)**

> Imagine que você está projetando um sistema de navegação para ambulâncias em uma cidade. Cada interseção é representada como um nó e cada rua como uma aresta ponderada com o tempo médio de deslocamento. Você precisa encontrar a rota mais rápida entre o hospital e o local de atendimento, garantindo que o caminho tenha custo mínimo e seja correto mesmo em grandes redes urbanas.

**Atividades**

- Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

- Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

Teste o algoritmo em um grafo de exemplo.

**Explique (Questões Discursivas):**

1. Por que Dijkstra exige arestas não negativas?

2. Qual a complexidade do algoritmo com lista de adjacência e heapq?

💡 Dica: Compare seu resultado com um mapa simples — se mudar o peso de uma rua, a rota muda?

In [ ]:
import heapq
import math

def dijkstra(graph, start):
    # Distâncias iniciais
    distances = {node: math.inf for node in graph}
    parents = {node: None for node in graph}
    distances[start] = 0

    pq = [(0, start)]  # (distância, nó)

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        # Se já existe distância melhor, ignora
        if current_distance > distances[current_node]:
            continue

        # Relaxamento
        for neighbor, weight in graph[current_node].items():
            new_dist = current_distance + weight

            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                parents[neighbor] = current_node
                heapq.heappush(pq, (new_dist, neighbor))

    return distances, parents


def reconstruct_path(parent, target):
    path = []
    current = target

    while current is not None:
        path.append(current)
        current = parent.get(current)

    return list(reversed(path))


# Exemplo
example_graph = {
    'Hospital': {'A': 3, 'B': 5},
    'A': {'Hospital': 3, 'C': 4, 'B': 2},
    'B': {'Hospital': 5, 'A': 2, 'D': 6},
    'C': {'A': 4, 'Atendimento': 5, 'D': 1},
    'D': {'B': 6, 'C': 1, 'Atendimento': 2},
    'Atendimento': {'C': 5, 'D': 2}
}

start_node = 'Hospital'
target_node = 'Atendimento'

distances, parents = dijkstra(example_graph, start_node)

if distances[target_node] == math.inf:
    print("Não existe caminho.")
else:
    path = reconstruct_path(parents, target_node)
    print("Custo mínimo:", distances[target_node])
    print("Caminho:", " -> ".join(path))

print("\nDistâncias:")
for node, dist in distances.items():
    print(f"{node}: {dist}")


**Por que Dijkstra exige arestas não negativas?**
Porque o algoritmo é "guloso" e assume que o caminho encontrado para um nó é definitivo. Se existisse uma aresta negativa, um caminho que parece mais longo poderia, no final, tornar-se mais curto, e o Dijkstra não saberia lidar com isso pois não revisita nós "fechados".

**Qual a complexidade (com heapq)?**
É $O(E \log V)$ (onde $E$ = arestas, $V$ = vértices). Isso ocorre porque as operações de heapq (inserir/remover) custam $O(\log V)$, e no pior caso, olhamos todas as $E$ arestas e as $V$ vértices.

**Parte B — A-Star (Busca Informada com Heurística Admissível)**

> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.

**Explique (Questões Discursivas):**

1. A* vs Dijkstra: qual expande menos nós?

2. Por que a heurística Manhattan é admissível nesse caso?

💡 Cenário real: O A* é amplamente usado em robôs aspiradores, drones e jogos. Seu desafio é aplicar o mesmo raciocínio.

In [ ]:
import numpy as np
import heapq
import math

# --- A* ---

def heuristic(a, b):
    # Distância Manhattan
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)


def a_star(grid, start, goal, h_func):
    rows, cols = grid.shape

    # Custos reais e estimados
    g_score = {(r, c): math.inf for r in range(rows) for c in range(cols)}
    f_score = {(r, c): math.inf for r in range(rows) for c in range(cols)}
    parents = {(r, c): None for r in range(rows) for c in range(cols)}

    g_score[start] = 0
    f_score[start] = h_func(start, goal)

    pq = [(f_score[start], start)]

    while pq:
        _, current = heapq.heappop(pq)

        # Chegou no objetivo
        if current == goal:
            return reconstruct_path_astar(parents, goal), g_score

        r, c = current
        neighbors = [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]

        for nr, nc in neighbors:
            # Fora do grid
            if not (0 <= nr < rows and 0 <= nc < cols):
                continue

            # Obstáculo
            if grid[nr][nc] == 1:
                continue

            tentative = g_score[current] + 1  # custo fixo por passo

            if tentative < g_score[(nr, nc)]:
                g_score[(nr, nc)] = tentative
                f_score[(nr, nc)] = tentative + h_func((nr, nc), goal)
                parents[(nr, nc)] = current
                heapq.heappush(pq, (f_score[(nr, nc)], (nr, nc)))

    # Não achou caminho
    return [], g_score


def reconstruct_path_astar(parent, target):
    path = []
    cur = target

    while cur is not None:
        path.append(cur)
        cur = parent.get(cur)

    return list(reversed(path))


# --- Teste simples ---

GRID_SIZE = 20
OBSTACLE_PROB = 0.15

grid = (np.random.rand(GRID_SIZE, GRID_SIZE) < OBSTACLE_PROB).astype(int)

start = (0, 0)
goal = (GRID_SIZE - 1, GRID_SIZE - 1)

grid[start] = 0
grid[goal] = 0

print(f"Buscando caminho de {start} até {goal}...")

path, costs = a_star(grid, start, goal, heuristic)

if not path:
    print("Nenhum caminho encontrado.")
else:
    print(f"Caminho achado! Custo: {costs[goal]}")

    # Visualização simples em texto
    viz = grid.copy().astype(str)
    viz[viz == '0'] = ' .'
    viz[viz == '1'] = '██'

    for (r, c) in path:
        if (r, c) == start:
            viz[r, c] = ' S'
        elif (r, c) == goal:
            viz[r, c] = ' E'
        else:
            viz[r, c] = ' *'

    print("\nMapa (S=Start, E=End, *=Path):")
    for row in viz:
        print("".join(row))


**A vs Dijkstra: qual expande menos nós?**

O A* expande muito menos nós. O Dijkstra explora "em todas as direções" (cego), enquanto o A* usa a heurística para focar a busca na direção do objetivo (informado).

**Por que a heurística Manhattan é admissível?**
Uma heurística é "admissível" se ela nunca superestima o custo real. A Distância Manhattan calcula o custo mínimo de passos no grid se não houvesse obstáculos. Como os obstáculos só podem aumentar o caminho real, a heurística é sempre menor ou igual ao custo verdadeiro.

**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem)**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

**Explique (Questões Discursivas):**

1. Em que situação cada tipo de percurso é mais indicado?

In [ ]:
# --- BST simples ---

class Node:
    def __init__(self, key):
        self.val = key
        self.left = None
        self.right = None


def insert(root, key):
    # Insere na BST
    if root is None:
        return Node(key)

    if key < root.val:
        root.left = insert(root.left, key)
    else:
        root.right = insert(root.right, key)

    return root


# --- Percursos ---

def in_order(root):
    if not root:
        return []
    return in_order(root.left) + [root.val] + in_order(root.right)


def pre_order(root):
    if not root:
        return []
    return [root.val] + pre_order(root.left) + pre_order(root.right)


def post_order(root):
    if not root:
        return []
    return post_order(root.left) + post_order(root.right) + [root.val]


# --- Teste ---

valores = [50, 30, 70, 20, 40, 60, 80, 35, 45]
root = None

for v in valores:
    root = insert(root, v)

print("--- Percursos ---")
print("In-order:   ", in_order(root))
print("Pre-order:  ", pre_order(root))
print("Post-order: ", post_order(root))


**Em que situação cada tipo de percurso é mais indicado?**

**Em-Ordem (In-order):** É o mais indicado quando se precisa obter os valores da árvore de forma ordenada. Em uma Árvore Binária de Busca (BST), a travessia em-ordem visita os nós em ordem crescente (do menor para o maior), sendo a forma canônica de listar os elementos ordenadamente.

**Pré-Ordem (Pre-order):** É ideal para copiar ou clonar uma árvore. Ao visitar a raiz primeiro, você pode criar o nó pai e, em seguida, chamar recursivamente a função para construir as sub-árvores esquerda e direita. Também é usado para salvar uma árvore em um arquivo, pois permite a reconstrução exata da estrutura original.

**Pós-Ordem (Post-order):** É fundamental para deletar ou liberar a memória de uma árvore. Você precisa visitar (e deletar) os filhos (esquerdo e direito) antes de poder deletar o nó pai. Outro uso comum é em calculadoras de notação polonesa reversa, onde você processa os operandos (filhos) antes de aplicar o operador (pai).

**Parte D — Reflexões (Respostas Curtas)**

Responda de forma argumentativa (5–10 linhas cada):

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?

2. Diferencie corretude e otimalidade nos algoritmos estudados.

3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.

4. Como heurísticas inconsistentes podem afetar o resultado do A*?

💬 Sugestão: use exemplos de mapas, jogos, sistemas de busca ou árvores sintáticas.

Insira suas respostas aqui

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?

O A* tem um custo extra (overhead) para calcular a heurística e gerenciar o f-score na fila de prioridade. Se o grafo for muito pequeno ou extremamente denso (muitas arestas), o Dijkstra pode ser comparável ou até mais rápido, pois a complexidade de heapq ($E \log V$) pode ser impactada. Além disso, se o objetivo for apenas encontrar qualquer caminho (não o ideal) e o tempo de processamento for crítico, um DFS ou BFS poderia ser mais simples.

2. Diferencie corretude e otimalidade nos algoritmos estudados.

Corretude (Correctness): Significa que o algoritmo funciona e entrega uma resposta válida. Por exemplo, um algoritmo de busca (como DFS/BFS) é correto se ele encontra um caminho entre o início e o fim, caso exista.Otimalidade (Optimality): Significa que o algoritmo não apenas encontra uma resposta, mas garante que é a melhor resposta possível. Dijkstra e A* (com heurística admissível) são otimais porque eles garantem encontrar o caminho de menor custo, e não apenas um caminho qualquer.

3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.

Em-ordem (In-order): Listar dados de forma ordenada em uma Árvore Binária de Busca. Ex: Imprimir um catálogo de produtos ordenado por preço (do mais barato ao mais caro) ou por nome (em ordem alfabética).Pré-ordem (Pre-order): Salvar a hierarquia de um sistema de arquivos ou de um documento (como XML/HTML). A pasta (nó pai) é listada antes de seu conteúdo (filhos), permitindo a reconstrução exata da estrutura.Pós-ordem (Post-order): Resolução de fórmulas matemáticas em notação polonesa reversa. Você precisa processar os operandos (filhos) antes de aplicar o operador (nó pai). Ex: 3 4 + (lê 3, lê 4, aplica o +).

4. Como heurísticas inconsistentes podem afetar o resultado do A*?

Uma heurística inconsistente (ou não-monotônica) quebra a regra de que o custo heurístico não deve diminuir mais do que o custo real do caminho. Isso torna o A* ineficiente. Embora ele ainda possa encontrar o caminho ótimo (contanto que a heurística seja admissível), ele perde sua maior vantagem: a garantia de que, uma vez que um nó é "fechado" (visitado), nunca mais precisará ser reaberto. Com uma heurística inconsistente, o algoritmo pode ser forçado a revisitar e re-expandir nós várias vezes, fazendo-o explorar muito mais do grafo, de forma similar ao Dijkstra.

**Entrega**

Crie um repositório público chamado ia-grafos-seu-nome.

Envie para o repositório o arquivo ia_grafos_buscas.ipynb.

Submeta o link do repositório no ambiente da disciplina **Portal Digital Fametro.**.

**Integridade Acadêmica**

1. O trabalho é individual.
2. Discussões conceituais são permitidas, mas o código deve ser inteiramente autoral.
3. Verificações de similaridade serão aplicadas a todas as submissões.
4. Busque e estude os algoritmos através de pesquisas na internet, livros, slides da disciplina.